**Evaluate the YOLO model**


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# The working path of this code file should be inside the yolo folder
from os import chdir, listdir
print(listdir('./'))

# chdir('/content/drive/MyDrive/Sem_3/ML/F20_ML_final_project/yolo/')
# print(listdir('./'))

['.ipynb_checkpoints', 'annotation_preprocess.ipynb', 'data', 'eval.ipynb', 'loss_data', 'map', 'nets', 'predict.ipynb', 'test_img', 'train.ipynb', 'utils', 'yolo.py', '__pycache__']


In [6]:
import os
import cv2
import torch
import colorsys
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from yolo import YOLO
from nets.yolo4 import YoloBody
import xml.etree.ElementTree as ET
from PIL import Image, ImageFont, ImageDraw
from utils.utils import non_max_suppression, bbox_iou, DecodeBox,letterbox_image,yolo_correct_boxes

In [7]:
# Generate the ground-truth

idx_to_label = ('without_mask', 'with_mask', 'mask_weared_incorrect')
gc_dir = './map/input/ground-truth/'
# gc_dir = './../yolo/map/input/ground-truth/'


test_img = []
print('Start generating ground-truth!')
with open('./data/test_data.txt', 'r') as f:
    test_data = f.read().split('\n')
    # print(test_dataset)
    for i in range(len(test_data)):
        if test_data[i].strip() == '':
            break
        t = test_data[i].split()
        img_path = t[0]
        objects = t[1:]
        test_img.append(img_path)
        with open(gc_dir + ('test_%03d.txt' % i), 'w') as f:
            for obj in objects:
                face_info = obj.split(',')
                f.write('%s %s %s %s %s\n' % (idx_to_label[int(face_info[-1])], face_info[0], face_info[1], face_info[2], face_info[3]))
        # if i % 10 == 9:
        #     print('Steps: %3d/%3d' % (i + 1, len(test_data)))
print('Finish generating ground-truth!')

Start generating ground-truth!
Finish generating ground-truth!


In [4]:
# making detection-results
class mAP_Yolo(YOLO):
    #---------------------------------------------------#
    #   检测图片
    #---------------------------------------------------#
    def detect_image(self, image_id, image):
        self.confidence = 0.05
        image_shape = np.array(np.shape(image)[0:2])

        crop_img = np.array(letterbox_image(image, (self.model_image_size[0], self.model_image_size[1])))
        photo = np.array(crop_img, dtype = np.float32)
        photo /= 255.0
        photo = np.transpose(photo, (2, 0, 1))
        photo = photo.astype(np.float32)
        images = []
        images.append(photo)
        images = np.asarray(images)

        with torch.no_grad():
            images = torch.from_numpy(images)
            if self.cuda:
                images = images.cuda()
            outputs = self.net(images)
            
        output_list = []
        for i in range(3):
            output_list.append(self.yolo_decodes[i](outputs[i]))
        output = torch.cat(output_list, 1)
        batch_detections = non_max_suppression(output, len(self.class_names),
                                                conf_thres=self.confidence,
                                                nms_thres=0.3)

        try:
            batch_detections = batch_detections[0].cpu().numpy()
        except:
            return image
            
        top_index = batch_detections[:,4]*batch_detections[:,5] > self.confidence
        top_conf = batch_detections[top_index,4]*batch_detections[top_index,5]
        top_label = np.array(batch_detections[top_index,-1],np.int32)
        top_bboxes = np.array(batch_detections[top_index,:4])
        top_xmin, top_ymin, top_xmax, top_ymax = np.expand_dims(top_bboxes[:,0],-1),np.expand_dims(top_bboxes[:,1],-1),np.expand_dims(top_bboxes[:,2],-1),np.expand_dims(top_bboxes[:,3],-1)

        # 去掉灰条
        boxes = yolo_correct_boxes(top_ymin,top_xmin,top_ymax,top_xmax,np.array([self.model_image_size[0],self.model_image_size[1]]),image_shape)

        with open('./map/input/detection-results/test_' + image_id + '.txt', 'w') as f:
            for i, c in enumerate(top_label):
                predicted_class = self.class_names[c]
                score = str(top_conf[i])

                top, left, bottom, right = boxes[i]
                f.write(' '.join((str(predicted_class), str(score), str(left), str(top), str(right), str(bottom))) + '\n')
    

print('Start detecting results!')

yolo = mAP_Yolo()

for i in range(len(test_img)):
    img = Image.open(test_img[i])
    yolo.detect_image('%03d' % i, img)
    print('[done]', i)

# for image in images:
#     img = Image.open(os.path.join(image_path, image))
#     yolo.detect_image(image.split('.')[0], img)
#     print('[done] ' + image)
print('Finish detecting results!')

Start detecting results!
Loading pretrained weights.
Finish loading!
./data/yolov4_test.pth model, anchors, and classes loaded.
(12, 4)
[done] 0
(5, 4)
[done] 1
(5, 4)
[done] 2
(1, 4)
[done] 3
(5, 4)
[done] 4
(1, 4)
[done] 5
(2, 4)
[done] 6
(1, 4)
[done] 7
(3, 4)
[done] 8
(3, 4)
[done] 9
(5, 4)
[done] 10
(4, 4)
[done] 11
(8, 4)
[done] 12
(1, 4)
[done] 13
(1, 4)
[done] 14
(1, 4)
[done] 15
(10, 4)
[done] 16
(2, 4)
[done] 17
(7, 4)
[done] 18
(6, 4)
[done] 19
(3, 4)
[done] 20
(10, 4)
[done] 21
(1, 4)
[done] 22
(5, 4)
[done] 23
(1, 4)
[done] 24
(10, 4)
[done] 25
(3, 4)
[done] 26
(1, 4)
[done] 27
(2, 4)
[done] 28
(5, 4)
[done] 29
(1, 4)
[done] 30
(1, 4)
[done] 31
(9, 4)
[done] 32
(1, 4)
[done] 33
(3, 4)
[done] 34
(4, 4)
[done] 35
(1, 4)
[done] 36
(1, 4)
[done] 37
(1, 4)
[done] 38
(2, 4)
[done] 39
(2, 4)
[done] 40
(8, 4)
[done] 41
(2, 4)
[done] 42
(1, 4)
[done] 43
(5, 4)
[done] 44
(11, 4)
[done] 45
(5, 4)
[done] 46
(2, 4)
[done] 47
(5, 4)
[done] 48
(10, 4)
[done] 49
(4, 4)
[done] 50
(4, 4)
[d